In [1]:
import os
import sys
import math
from pathlib import Path
from typing import Optional, Dict, Tuple

import numpy as np
import cv2

def setup_project_path() -> Path:
    current = Path.cwd()
    while current != current.parent and not (current / "craft").exists():
        current = current.parent
    if not (current / "craft").exists():
        raise RuntimeError("Could not find project_root containing 'craft' directory.")
    return current

project_root = setup_project_path()
sys.path.insert(0, str(project_root))


In [2]:
def imwrite_unicode(path, img):
    path = str(path)
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True

def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    path = str(path)
    try:
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None


In [3]:
def extract_bbox_xyxy(
    img_bgr_or_gray: np.ndarray,
    threshold: int = 0
) -> Optional[Tuple[int, int, int, int]]:
    if img_bgr_or_gray is None:
        return None

    if img_bgr_or_gray.ndim == 3:
        gray = cv2.cvtColor(img_bgr_or_gray, cv2.COLOR_BGR2GRAY)
    else:
        gray = img_bgr_or_gray.copy()

    mask = (gray > threshold)
    ys, xs = np.where(mask)

    if xs.size == 0:
        return None

    x0, x1 = int(xs.min()), int(xs.max())
    y0, y1 = int(ys.min()), int(ys.max())
    return (x0, y0, x1, y1)


def xyxy_to_corners(x0: int, y0: int, x1: int, y1: int):
    return {
        "TL": (x0, y0),
        "TR": (x1, y0),
        "BR": (x1, y1),
        "BL": (x0, y1),
    }


In [4]:
def write_bbox_txt_with_text_map(
    folder: Path,
    out_txt: Path,
    text_map: Dict[str, str],
    threshold: int = 0,
    read_flags: int = cv2.IMREAD_UNCHANGED
) -> None:
    exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
    images = [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts]
    images.sort(key=lambda p: p.name)

    out_txt.parent.mkdir(parents=True, exist_ok=True)

    original_text = text_map.get(folder.name, "(missing)")

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(f"# Image Name: {folder.name}\n")
        f.write(f"# TEXT: {original_text}\n")
        f.write("-" * 60 + "\n")

        for p in images:
            img = imread_unicode(p, flags=read_flags)
            xyxy = extract_bbox_xyxy(img, threshold=threshold)

            f.write(f"file={p.name}\n")
            if xyxy is None:
                f.write("  INVALID (no ink pixels)\n")
            else:
                x0, y0, x1, y1 = xyxy
                corners = xyxy_to_corners(x0, y0, x1, y1)
                f.write(f"  TL={corners['TL']}\n")
                f.write(f"  TR={corners['TR']}\n")
                f.write(f"  BR={corners['BR']}\n")
                f.write(f"  BL={corners['BL']}\n")
            f.write("-" * 60 + "\n")


In [5]:
def run_bbox_batch_with_text_map(
    in_root: Path,
    out_root: Path,
    text_map: Dict[str, str],
    threshold: int = 0,
    read_flags: int = cv2.IMREAD_UNCHANGED
) -> None:
    in_root = Path(in_root)
    out_root = Path(out_root)
    out_root.mkdir(parents=True, exist_ok=True)

    subfolders = [p for p in in_root.iterdir() if p.is_dir()]
    subfolders.sort(key=lambda p: p.name)

    for folder in subfolders:
        out_txt = out_root / f"{folder.name}.txt"
        write_bbox_txt_with_text_map(
            folder=folder,
            out_txt=out_txt,
            text_map=text_map,
            threshold=threshold,
            read_flags=read_flags
        )


In [6]:
text_map = {
    "test1": "바른글씨",
    "test2": "캡스톤디자인",
    "test3": "숭실대학교",
    "test4": "기말 시험",
    "test5": "소프트웨어 분석",
    "test6": "안녕하세요",
    "test7": "숭실대학교",
    "test8": "안녕하세요"
}

in_root = project_root / "characters"
out_root = project_root / "results" / "char_bbox"

run_bbox_batch_with_text_map(
    in_root=in_root,
    out_root=out_root,
    text_map=text_map,
    threshold=0,
    read_flags=cv2.IMREAD_UNCHANGED
)

print("DONE")
print("Input :", in_root)
print("Output:", out_root)


DONE
Input : D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\characters
Output: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\results\char_bbox
